In [1]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import numpy as np

In [2]:
import ampligraph
# Benchmark datasets are under ampligraph.datasets module
from ampligraph.datasets import load_fb15k_237
# load fb15k-237 dataset
dataset = load_fb15k_237()

## Train and predict scores

In [3]:
# Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel

# create the model with transe scoring function
partitioned_model = ScoringBasedEmbeddingModel(eta=2, 
                                               k=50, 
                                               scoring_type='TransE')
partitioned_model.compile(optimizer='adam', loss='multiclass_nll')


partitioned_model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                      batch_size=10000, 
                      use_partitioning=True, # set flag to partition the inputs
                      epochs=10)


_split: memory before: 864.0Bytes, after: 13.002MB, consumed: 13.001MB; exec time: 138.12s
Epoch 1/10
32/32 [==============================] - 9s 293ms/step - loss: 9643.8496
Epoch 2/10
32/32 [==============================] - 8s 251ms/step - loss: 9608.9883
Epoch 3/10
32/32 [==============================] - 8s 249ms/step - loss: 9564.9219
Epoch 4/10
32/32 [==============================] - 8s 249ms/step - loss: 9509.7002
Epoch 5/10
32/32 [==============================] - 8s 252ms/step - loss: 9443.8955
Epoch 6/10
32/32 [==============================] - 8s 249ms/step - loss: 9366.0791
Epoch 7/10
32/32 [==============================] - 8s 250ms/step - loss: 9276.2832
Epoch 8/10
32/32 [==============================] - 8s 249ms/step - loss: 9176.0684
Epoch 9/10
32/32 [==============================] - 8s 252ms/step - loss: 9067.4639
Epoch 10/10
32/32 [==============================] - 8s 250ms/step - loss: 8952.7305


In [4]:
# Unfiltered evaluation
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                   batch_size=400)

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)



28 triples containing invalid keys skipped!
53/53 [==============================] - 216s 4s/step


(964.4990948233682, 0.08503310959927467, 0.0, 0.22903415206967415, 20438)

In [5]:
# Filtered evaluation
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                        batch_size=400,
                        corrupt_side='s,o',
                        use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                              'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                              'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)



28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!
53/53 [==============================] - 398s 8s/step


(835.8881984538605,
 0.1846081325238635,
 0.1265779430472649,
 0.30357667090713375,
 20438)

In [6]:
from ampligraph.utils import save_model
save_model(model=partitioned_model, model_name_path='./partitioned_model')

The path ./partitioned_model already exists. This save operation will overwrite the model                 at the specified path.
WARNING - Found untraced functions such as _get_ranks while saving (showing 1 of 1). These functions will not be directly callable after loading.


In [7]:
from ampligraph.utils import restore_model
model = restore_model('./partitioned_model')

In [8]:
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                        batch_size=400,
                        corrupt_side='s,o',
                        use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                              'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                              'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)



28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!
53/53 [==============================] - 403s 8s/step


(835.8881984538605,
 0.1846081325238635,
 0.1265779430472649,
 0.30357667090713375,
 20438)